## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [1]:
# File location and type
file_location = "Advertising.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

NameError: name 'spark' is not defined

In [ ]:
# Create a view or table

temp_table_name = "Advertising_csv"

df.createOrReplaceTempView(temp_table_name)

In [ ]:
%sql

/* Query the created temp table in a SQL cell */

select * from `Advertising_csv`

In [ ]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "Advertising_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [ ]:
regressionDataFrame = df.drop('_c0')

In [ ]:
regressionDataFrame.take(10)

In [ ]:
regressionDataFrame.show(10)

In [ ]:
regressionDataRDD = regressionDataFrame.rdd.map(list)

In [ ]:
regressionDataRDD.take(5)

In [ ]:
from pyspark.mllib.regression import LabeledPoint
regressionDataLabelPoint = regressionDataRDD.map(lambda data : LabeledPoint(data[3], data[0:3]))

In [ ]:
regressionDataLabelPoint.take(5)

In [ ]:
# train test splitting
regressionLabelPointSplit = regressionDataLabelPoint.randomSplit([0.7, 0.3])

In [ ]:
regressionLabelPointTrainData = regressionLabelPointSplit[0]

In [ ]:
type(regressionLabelPointTrainData)

In [ ]:
regressionLabelPointTrainData.toDF().toPandas()

In [ ]:
regressionLabelPointTrainData.toDF().toPandas()['features'].values

In [3]:
# alternative to using Pysparks's own ml lib, you can convert to Pandas and go back to using sklearn
# you will also need convert DenseVector object from PySpark, and cast as a numpy array
# however the computational resources needed to store your data will no be distributed either! so please keep that in mind

from sklearn.linear_model import LinearRegression
X = regressionLabelPointTrainData.toDF().toPandas()['features'].values
y = regressionLabelPointTrainData.toDF().toPandas()['label'].values.reshape(-1,1)
# print(X)
lr = LinearRegression()
lr.fit(X,y)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
regressionLabelPointTrainData.take(5)

In [ ]:
regressionLabelPointTestData = regressionLabelPointSplit[1]

In [ ]:
regressionLabelPointTestData.take(5)

In [ ]:
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

In [ ]:
model = lr.fit(regressionLabelPointTrainData)

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD as lrSGD

ourModelWithLinearRegression  = lrSGD.train(data = regressionLabelPointTrainData, 
                                            iterations = 100, step = 0.0002, intercept = True)

In [ ]:
ourModelWithLinearRegression